# Setup

Please ensure you have imported a HF API key.

After doing so, please run the setup cell below.

In [ ]:
!pip install git+https://github.com/huggingface/transformers

# Generated Code

In [ ]:
!pip install -U "bitsandbytes"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token= user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

In [ ]:
import torch
from transformers import AutoTokenizer, Gemma3ForCausalLM, BitsAndBytesConfig

ckpt = "google/gemma-3-4b-it"
tokenizer = AutoTokenizer.from_pretrained(ckpt)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 # Or torch.float16
)

model = Gemma3ForCausalLM.from_pretrained(
    ckpt,
    torch_dtype = torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

# Initialization

In [ ]:
import json
function_definitions_list = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather conditions for a specific location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g., San Francisco, CA"
                }
            },
            "required": ["location"]
        }
    },
    {
        "name": "send_message",
        "description": "Sends a message to a recipient.",
        "parameters": {
             "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The email address or identifier of the recipient."
                },
                "body": {
                    "type": "string",
                    "description": "The content of the message."
                }
            },
            "required": ["recipient", "body"]
        }
    }
]

function_definitions_json_string = json.dumps(function_definitions_list, indent=2)




## Parser

In [ ]:
import json
import re

def parse_tool_calls(decoded_output: str) -> list | None:
    cleaned_output = decoded_output.strip()
    match = re.search(r"```json\s*(\{.*?\})\s*```", cleaned_output, re.DOTALL | re.IGNORECASE)
    if match:
        json_string = match.group(1)
    else:
        json_string = cleaned_output.replace("<end_of_turn>", "").strip()

    if not json_string.startswith('{') or not json_string.endswith('}'):
        return None

    try:
        parsed_json = json.loads(json_string)
    except json.JSONDecodeError as e:
        print(f"Parser Error: Failed to decode JSON: {e}")
        return None

    if isinstance(parsed_json, dict) and "function_calls" in parsed_json:
        function_calls_list = parsed_json["function_calls"]
        if isinstance(function_calls_list, list):
            validated_calls = []
            for call in function_calls_list:
                if isinstance(call, dict) and "name" in call and "parameters" in call:
                    validated_calls.append(call)
                else:
                    print(f"Parser Error: Invalid item structure in function_calls list: {call}")
                    return None
            return validated_calls
        else:
            print("Parser Error: 'function_calls' key did not contain a list.")
            return None
    else:
        return None

## Parsing function calls

## Defining functions

In [ ]:
def get_current_weather(location: str) -> str:
    """Gets the current weather for a location."""
    print('Checked weather via function')
    if "new york" in location.lower():
        return json.dumps({"location": location, "temperature": "95F", "condition": "Sunny"})
    elif "san francisco" in location.lower():
         return json.dumps({"location": location, "temperature": "60F", "condition": "Foggy"})
    else:
        return json.dumps({"location": location, "temperature": "80F", "condition": "Rainy"})

def send_message(recipient: str, body: str) -> str:
    """Sends a message to a recipient."""
    print('Sent message via function')
    return json.dumps({"status": "Message sent successfully", "to": recipient})

In [ ]:
available_tools = {
    "get_current_weather": get_current_weather,
    "send_message": send_message,
}

## Executing

In [ ]:
def execute_tool_calls(parsed_tool_calls: list, tool_registry: dict) -> list:
    """
    Executes a list of tool calls based on the parsed output and a registry.

    Args:
        parsed_tool_calls: The list of dicts from the parser, e.g.,
                           [{"name": "...", "parameters": {...}}, ...].
                           Assumes this is not None (checked before calling).
        tool_registry: A dictionary mapping tool names (str) to callable functions.

    Returns:
        A list of results from executing each tool call. Each result is often
        stored as a dict containing the original call info and the output.
        Returns an empty list if the input list was empty.
    """
    execution_results = []

    if not parsed_tool_calls:
        return execution_results

    for tool_call in parsed_tool_calls:
        function_name = tool_call.get("name")
        parameters = tool_call.get("parameters", {})

        if not function_name:
            print("Executor Error: Tool call missing 'name'. Skipping.")
            execution_results.append({
                "call": tool_call,
                "error": "Missing function name"
            })
            continue

        if function_name not in tool_registry:
            print(f"Executor Error: Tool '{function_name}' not found in registry. Skipping.")
            execution_results.append({
                "call": tool_call,
                "error": f"Function '{function_name}' not registered."
            })
            continue

        function_to_call = tool_registry[function_name]

        try:
            result = function_to_call(**parameters)
            execution_results.append({
                "call": tool_call,
                "output": result
            })
            print(f"Executor: Call to {function_name} succeeded. Result: {result}")

        except TypeError as e:
            print(f"Executor Error: TypeError calling {function_name}: {e}. Check parameters.")
            execution_results.append({
                "call": tool_call,
                "error": f"Parameter mismatch for '{function_name}': {e}"
            })
        except Exception as e:
            print(f"Executor Error: Exception during execution of {function_name}: {e}")
            execution_results.append({
                "call": tool_call,
                "error": f"Execution error in '{function_name}': {e}"
            })

    return execution_results

# Testing

In [ ]:
MAX_TURNS = 5
turn_count = 0

setup_instructions = """
You are a highly competent and professional assistant designed to answer user queries, utilizing specialized tools when necessary. Follow these steps meticulously:

**Core Principle: Mandatory Tool Use**
If any part of the user's request, or any necessary follow-up action identified after receiving tool results, **directly matches the capability of an available tool** (e.g., the user asks to "send a message" and a `send_message` tool exists), you **MUST** use that tool by invoking it via the JSON format in Step 3. **Do not simply generate text describing the action or its outcome if a tool exists to perform that action.**

**Step 1: Analyze the User Query**
   - Understand the user's goal and the information requested.
   - Determine if any available tools are required based on the user's explicit request or implied necessary actions.

**Step 2: Decide Action**
   - **If no tools are required:** Proceed directly to Step 5.
   - **If one or more tools ARE required (considering the Mandatory Tool Use principle):** Proceed to Step 3.

**Step 3: Invoke Tools (If Required)**
   - Identify the necessary tool(s) and parameters.
   - **CRITICAL:** Respond ONLY with the required function calls in the following strict JSON format:
     ```json
     { "function_calls": [ {"name": "function_name", "parameters": { ... }}, ... ] }
     ```
   - **DO NOT include ANY other text, greetings, or explanations outside this JSON structure when invoking tools.** Stop your response immediately after the closing brace `}` of the JSON object.

**Step 4: Process Tool Results**
   - After you output the function calls (Step 3), you will receive the results back in the next user message (e.g., "Result for the call to [tool_name]: [result data]").
   - Carefully analyze these results.
   - **Determine if the results sufficiently fulfill the user's original query OR if additional tool calls are needed based on these results AND the Mandatory Tool Use principle.** (For example, if the original query was "Get X and then do Y with it", and you just got X, you must now check if doing Y requires a tool).
     - If the original query is now fully answered by the results and no further tool actions are mandated, proceed to Step 5.
     - If more tool calls are needed, go back to Step 3.

**Step 5: Generate Final Response**
   - **Synthesize a comprehensive, user-friendly, natural language answer.**
   - If tools were used (you completed Step 4), formulate the answer *based on the final results* you received. Ensure the answer accurately reflects the outcome of all tool actions performed.
   - If no tools were ever needed (you skipped from Step 2), formulate the answer based directly on the user query.
   - **CRITICAL:** Your final response in this step MUST be plain text. **DO NOT output `{"function_calls": []}` or any other JSON when providing the final answer.**

"""

user_query = "check weather of odisha and send message to ashish wear clothes according to that weather"

final_user_content = f"""{setup_instructions}

Available Tools:
{function_definitions_json_string}

User Query: {user_query}"""


chat_history = [
    {"role": "user", "content": final_user_content}
]


while turn_count < MAX_TURNS:
    turn_count += 1
    print(f"\n--- Agent Turn {turn_count} ---")

    model_inputs = tokenizer.apply_chat_template(
        chat_history,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt"
    ).to(model.device)

    input_len = model_inputs.shape[1]

    with torch.inference_mode():
        generation = model.generate(input_ids=model_inputs, max_new_tokens=200, do_sample=False)
        new_generation_ids = generation[0][input_len:]
        decoded = tokenizer.decode(new_generation_ids, skip_special_tokens=True)

    print(f"Model Output (Turn {turn_count}):\n{decoded}")

    chat_history.append({"role": "assistant", "content": decoded})

    parsed_calls = parse_tool_calls(decoded)

    if parsed_calls:
        tool_results = execute_tool_calls(parsed_calls, available_tools)
        print("\n--- Tool Execution Results (Structured) ---")
        print(json.dumps(tool_results, indent=2))
        
        feedback_parts = []
        for result_info in tool_results:
            func_name = result_info.get("call", {}).get("name", "Unknown Function")
            output = result_info.get("output")
            error = result_info.get("error")
        
            if output:
                data_str = output
                try:
                    parsed_output = json.loads(output)
                    data_str = json.dumps(parsed_output)
                except:
                    pass # Keep raw output if not JSON
                feedback_parts.append(f"Result for the call to {func_name}: {data_str}")
            elif error:
                feedback_parts.append(f"Error during call to {func_name}: {error}")
        feedback_content = "\n".join(feedback_parts)
        
        print(f"\n--- Plain Text Feedback to Model ---\n{feedback_content}")
        
        # Append this plain text string to the history
        chat_history.append({"role": "user", "content": feedback_content}) # NO json.dumps here!


    else:
        print("\n--- No Function Calls Identified: Assuming Final Answer ---")
        final_response_to_user = decoded
        break

import re
import json

if turn_count >= MAX_TURNS:
    print("\n--- Reached Max Turns ---")
    final_response_to_user = chat_history[-1]['content'] if chat_history[-1]['role'] == 'assistant' else "Reached max turns without final answer."


cleaned_response = final_response_to_user

if cleaned_response: 
    pattern = r"(?:```json\s*)?\{\s*\"function_calls\"\s*:\s*\[.*?\]\s*\}(?:\s*```)?"

    cleaned_response = re.sub(pattern, '', cleaned_response, flags=re.DOTALL | re.IGNORECASE)

    cleaned_response = cleaned_response.strip()

    if not cleaned_response and final_response_to_user:
         print("Info: Regex cleaning removed the entire message. Using default.")
         cleaned_response = "Okay, I have completed the requested actions."

print("\n--- Final Response to User (Cleaned) ---")
print(cleaned_response)